<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/Jax-Journey/blob/main/making_functions_pure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Making Purified Versions of Impure Functions

image.png

In the class below, 

* ```params``` acts as a simple placeholders for what would be sent to ```pure_wrapped_user_fn``` later on. 

* ```self.params``` over-rides and hides the original class varibale ```params```.

* The ```self.get_param``` is just a protection to ensure that the end user of the class always accesses the class' functionality by passing ```params``` within ```pure_wrapped_user_fn```. If the user would not make mistakes(i.e., not call ```impure_user_fn``` directly), the ```get_param``` thing can be entirely eliminated and we can directly access ```self.param['w']``` etc. in the ```impure_user_fn```. 

* Notice how ```self.params``` and ```self.is_running_pure``` are set only when we enter the pure function, and reset while leaving it. This behavior will be seen in Flax too, later on.. where you won't even be able to access submodules, variables etc. you register in the ```setup()``` function using calls  like ```print(model.variable)```, outside of the class. 

In [ ]:
class PurifyingRegressor():

    # This is the function the user wants to write.
    def impure_user_fn(self, x):
        w = self.get_param('w')
        b = self.get_param('b')
        return w * x + b

    # This is the wrapping/mode logic.
    params = {}                                                                 
    is_running_pure = False

    def get_param(self, name):
        if not self.is_running_pure:
            raise Exception("We can only call this when wrapped!")
        else:
            return self.params[name]

    # This is the function that we want JAX to use!
    # Note how it defers all calculation and logic to
    # the user-written and just provides *context*!
    def pure_wrapped_user_fn(self, params, *args):
        self.is_running_pure = True
        self.params = params                                        
        result = self.impure_user_fn(*args)
        self.is_running_pure = False
        self.params = {}
        return result

In [ ]:
pure_predict_fn = PurifyingRegressor().pure_wrapped_user_fn

# Show that it's equal to this manually purified function:
manually_pure_predict_fn = lambda params, x: params['w'] * x + params['b']

args = ({'w': 13., 'b': 0.}, 42.)

assert pure_predict_fn(*args) == manually_pure_predict_fn(*args)

In [ ]:
x = PurifyingRegressor()

In [ ]:
x.pure_wrapped_user_fn(*args)

In [ ]:
print(x.params)

In [ ]:
import jax
f = jax.grad(pure_predict_fn)
g = jax.grad(manually_pure_predict_fn)

In [ ]:
print(f(*args))
print(g(*args))

{'b': array(1., dtype=float32), 'w': DeviceArray(42., dtype=float32)}
{'b': array(1., dtype=float32), 'w': DeviceArray(42., dtype=float32)}


# ~~Storing~~ Initializing Parameters Inside Class

* ```method``` is the user defined impure functions that accesses parameters, assuming they are members of the class.

* Whenever ```initial_params()``` function will be called, with a ```method```, and the ```method```'s arguments, every time ```method``` tries to access a ```param```, using ```self.get_param```, the parameter will be initialized. The ```method``` can send in the shape argument as the shape of result of some of its previous computation or one of its ```args```'s shape. The initializers can be specified beforehand or as an argument. So all the parameters will be initialized as we go on doing calculations and calling ```get_params()``` inside  ```method()``` [**which is running inside ```initial_params()```**].

* The ```purify_method``` function, returns a pure version of ```method``` that takes in ```params``` as its first argument, always. These are the same ```params``` which were returned by ```initial_params()```.

* Notice, how in this case we can't replace all ```self.get_param()``` with simple ```self.param[]``` in whatever ```method``` maybe, because we want our class to handle all initialization, retrieval etc.

In [ ]:
class FlatFancyNNModule():
    def __init__(self, name):
        self.name = name
        self.params = None
        self.are_we_pure = False
        self.are_we_initializing = False

    # This is where, depending on mode, we'll give the user-written
    # impure function different things!
    def get_param(self, name, shape, initializer):
        if self.are_we_initializing:
            if name not in self.params:
                self.params[name] = initializer(shape)
            return self.params[name]
        elif self.are_we_pure:
            return self.params[name]
        else:
            raise Exception("Can't access parameters outside of context!")

    # This function calls the initializers to give us an initial params dict.    
    def initial_params(self, method, *args):
        self.are_we_initializing = True
        self.params = {}
        method(*args)
        self.are_we_initializing = False
        x = self.params
        self.params = {}
        return x

    # This function returns a pure version of the function.
    def purify_method(self, method):
        def pure_method(params, *args):
            self.are_we_pure = True
            self.params = params
            result = method(*args)
            self.are_we_pure = False
            self.params = None
            return result
        return pure_method

In the below code, 
* The third argument to ```self.get_param``` is the initializer function.

* Initializer functions usually take in shape and rng key, but here, since our base class only has functionality for shape, we send in a function that only takes ```shape``` as argument.

* The initializers will assign 13 to ```w``` and 0 to ```b``` , irrespective of the shape of input. But their values as well as shapes can be made to depend on ```x``` 's value/shape. 

* Below, ```obj``` refers to an object of class ```FancyLinearRegressor```.

* Note here that both the calls ```obj.initial_params(predict, x)``` and ```obj.initial_params(rms,xs,ys)``` will initialise parameters correctly. Similarly, both the functions can be purified. 

* Outside the class, calls will never be made to these function(```predict``` and ```rms```) directly. They will be made to ```obj.initial_params()```, ```obj.purify_method()``` and to the purified method returned by ```obj.initial_params()```.

* ```obj.initial_params()``` will return ```params``` that must be stored in a variable outside class which must be used when calling the purified versions of ```rms()``` or ```predict()``` . 


In [ ]:
class FancyLinearRegressor(FlatFancyNNModule):
    def __init__(self, name="linreg"):
        super().__init__(name=name)
    
    def predict(self, x):
        # Our "initializers" are a bit simple here.
        w = self.get_param('w', (0,), lambda _shape: 13.)                       
        b = self.get_param('b', (0,), lambda _shape: 0.)
        return w * x + b

    def rms(self, xs, ys):
        # Same as before.
        w = self.get_param('w', (0,), lambda _shape: 13.)
        b = self.get_param('b', (0,), lambda _shape: 0.)
        return jnp.sqrt(jnp.sum(jnp.square(w * xs + b - ys)))

In [ ]:
my_regressor = FancyLinearRegressor()

sample_input = 999.  # a placeholder, necessary so the method can be run completely
params = my_regressor.initial_params(my_regressor.predict, sample_input)

print("Params:", params)

pure_predict_fn = my_regressor.purify_method(my_regressor.predict)

pure_predict_fn(params, 42.)

In [ ]:
def update_combiner(param, grad, lr=0.002):
    return param - lr * grad

In [ ]:
pure_rms_fn = my_regressor.purify_method(my_regressor.rms)
grad_rms_fn = jax.grad(pure_rms_fn)

params = my_regressor.initial_params(my_regressor.predict, sample_input)

for _ in range(15):
    print(pure_rms_fn(params, xs, ys))
    grads = grad_rms_fn(params, xs, ys)
    params = jax.tree_multimap(update_combiner, params, grads)

pure_predict_fn(params, 42.)

Suppose now, we now want to make a Neural Net, by composing together ```LinearRegressor```s. In the NN class, we will define various ```LinearRegressor```. 

\\
**The parameters of these objects can be initialised only when ```initial_params``` of these objects called.** 

\\
Suppose the ```predict``` function of NN, directly calls predict function of the ```LinearRegressor```s defined inside NN. 

\\
So, now, when we try to run ```NN_obj.initial_params(NN_obj.predict, sample_input)```, it will call ```NN_obj.predict(sample_input)``` which will, in turn call ```LinearRegressor.predict(sample_input)``` which will in turn call ```get_param()``` which will raise an exception because, although ```are_we_initializing``` of the Neural Network object is ```True```, it is ```False``` for the ```LinearRegressor```. 

\\
So, the correct way to proceed would be to initilaize and purify all submodules first,  store there parameters with yourself , then call your your own ```initial_param``` , and use only the purified versions of the member functions of submodules in your NN class. 

## Modifying NNModule to handle nested submodules

In [ ]:
class FancyNNModule():
    def __init__(self, name):
        self.name = name
        self.params = None
        self.are_we_pure = False
        self.are_we_initializing = False
        # Implementations should register their submodules here!
        self.submodules = {}

    # This is where, depending on mode, we'll give the user-written
    # impure function different things!
    def get_param(self, name, shape, initializer):
        if self.are_we_initializing:
            if name not in self.params:
                self.params[name] = initializer(shape)
            return self.params[name]
        elif self.are_we_pure:
            return self.params[name]
        else:
            raise Exception("Can't access parameters outside of context!")

    def set_are_we_pure(self, ispure):
        if ispure: self.params = {}
        self.are_we_pure = ispure
        for sm in self.submodules.values():
            sm.set_are_we_pure(ispure)
        if not ispure: self.params = None

    def set_are_we_initializing(self, isinit):
        if isinit:
            self.params = {}
        self.are_we_initializing = isinit
        for sm in self.submodules.values():
            sm.set_are_we_initializing(isinit)

    # This method gathers params from this module and all submodules.
    def gather_params(self):
        params = self.params
        for sm_name, sm in self.submodules.items():
            for name, value in sm.gather_params().items():
                params[sm_name + "/" + name] = value
        return params

    # This method spreads out params into self and all submodules.
    def spread_params(self, params):
        for name, value in params.items():
            path = name.split("/")
            if len(path) == 1:
                self.params[name] = value
            else:
                self.submodules[path[0]].spread_params({"/".join(path[1:]): value})

    # This function calls the initializers to give us an initial params dict.    
    def initial_params(self, method, *args):
        self.set_are_we_initializing(True)                                      
        method(*args)
        self.set_are_we_initializing(False)
        params = self.gather_params()
        return params

    # This function returns a pure version of the function.
    def purify_method(self, method):
        def pure_method(params, *args):
            self.set_are_we_pure(True)
            self.spread_params(params)
            result = method(*args)
            self.set_are_we_pure(False)
            return result
        return pure_method

We will understand the code above and below together. Note that each module is now required to maintain a dictionary of its immediate submodules(see line 38 below).

**Initialization Steps(Line 54-59 above)**

* Set ```are_we_initializing=True``` for the module, and all its submodules recursively.

* Call the ```method``` supplied in ```initial_params()``` using the args given. 

* Each call to ```get_param``` directly in the module(e.g., in ```FancyLSTMLM```) will lead to new parameters being assigned to the module's parent(```FancyNNModule```) class' ```params``` attribute.

* Each call to a submodule(e.g. to ```self.cell()``` in ```FancyLSTMLM```), will lead to a call to a member function of the submodule; and the ```get_param()``` calls within it will assign new parameters to that object's ```params``` attribute(i.e., to ```self.cell.params``` in the below example). 

* Then we reset ```are_we_initialising``` of all submodules recursively and gather all parameters of all the nested modules as done in ```gather_params()``` and return the gathered parameters. 

* Each module's ```self.params``` attribute consists of a dictionary whose keys are their direct parameters and values are the values of those parameters.

* ```gather_params()``` gathers together all those ```self.params``` into a ```params``` dict. The keys of this dicts are the paths(e.g. ```lstmcell/weight_ih``` would be a key in the ```param``` when calling ```gather_params()``` in ```FancyLSMTLM``` ) to each parameter in each of the submodules(nested as well as direct) and values are the values given to them after initialization. 

Similarly you can understand the ```purify_method()``` function's working. Observe how both ```spread_params()``` and ```get_params()``` need to recursively call **themselves** only. 



In [ ]:
random_key = jax.random.PRNGKey(0)

def unif_initializer(shape):
    global random_key
    sample_key, random_key = jax.random.split(random_key)
    return jax.random.uniform(sample_key, shape)

class FancyLSTMCell(FancyNNModule):
    def __init__(self, hid_dim, name="lstmcell"):
        super().__init__(name=name)
        self.hid_dim = hid_dim

    def __call__(self, inputs, h, c):
        weight_ih = self.get_param("weight_ih",
                (4*self.hid_dim, self.hid_dim), unif_initializer)
        weight_hh = self.get_param("weight_hh",
                (4*self.hid_dim, self.hid_dim), unif_initializer)
        bias = self.get_param("bias",
                (4*self.hid_dim,), lambda shape: jnp.zeros(shape))

        ifgo = weight_ih @ inputs + weight_hh @ h + bias
        i, f, g, o = jnp.split(ifgo, indices_or_sections=4, axis=-1)
        i = jax.nn.sigmoid(i)
        f = jax.nn.sigmoid(f)
        g = jnp.tanh(g)
        o = jax.nn.sigmoid(o)
        new_c = f * c + i * g
        new_h = o * jnp.tanh(new_c)
        return (new_h, new_c)

class FancyLSTMLM(FancyNNModule):
    def __init__(self, vocab_size, dim, name="lstmlm"):
        super().__init__(name=name)
        self.vocab_size = vocab_size
        self.dim = dim
        # Now create a submodule and register it!
        self.cell = FancyLSTMCell(dim)
        self.submodules[self.cell.name] = self.cell
    
    @property
    def hc_0(self):
        _c_0 = self.get_param("c_0",
                (self.dim,), lambda shape: jnp.zeros(shape))
        return (jnp.tanh(_c_0), _c_0)

    def forward(self, seq, hc):
        loss = 0.
        embeddings = self.get_param("embeddings",
                (self.vocab_size, self.dim), unif_initializer)
        for idx in seq:
            loss -= jax.nn.log_softmax(embeddings @ hc[0])[idx]
            hc = self.cell(embeddings[idx,:], *hc)
        return loss, hc

    def greedy_argmax(self, hc, length=6):
        idxs = []
        embeddings = self.get_param("embeddings",
                (self.vocab_size, self.dim), unif_initializer)
        for i in range(length):
            idx = jnp.argmax(embeddings @ hc[0])
            idxs.append(int(idx))
            hc = self.cell(embeddings[idx,:], *hc)
return idxs

In [ ]:
lm = FancyLSTMLM(vocab_size, 17)

# Since jitting and our mutable-state-galore "tracer" don't play nice, only
# call jit on the outermost level: the actually pure function!
pure_sample_fn = lm.purify_method(
    lambda: lm.greedy_argmax(lm.hc_0))
pure_forward_fn = jax.jit(lm.purify_method(
    lambda seq, hc: lm.forward(seq, hc if hc else lm.hc_0)))
grad_loss_fn = jax.jit(jax.grad(lm.purify_method(
    lambda seq, hc: lm.forward(seq, hc if hc else lm.hc_0)[0])))

params = lm.initial_params(lambda: lm.forward(jnp.array([0]), lm.hc_0))
print("All parameters, recursively found:", list(params.keys()))

print("Sample before:", pure_sample_fn(params))

bptt_length = 3
for epoch in range(101):
    totalloss = 0.
    hc = None
    for start in range(0, len(training_data), bptt_length):
        batch = training_data[start:start+bptt_length]
        loss, new_hc = pure_forward_fn(params, batch, hc)
        if epoch % 50 == 0:
            totalloss += loss.item()
        grads = grad_loss_fn(params, batch, hc)
        params = jax.tree_multimap(update_combiner_01, params, grads)
        hc = new_hc
    if totalloss:
        print("Loss:", totalloss)

print("Sample after:", pure_sample_fn(params))